In [1]:
import numpy as np
import scipy.optimize
import trfp

In [30]:
def __multipole(order, skew, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given multipole parameters
    and position.
    """
    # Takes a multipole strength A normalized to 4.5 cm
    r_pos = np.sqrt(x_pos**2 + y_pos**2)
    theta = np.arctan2(y_pos, x_pos)
    if skew == 0:
        b_magnitude = strength * (r_pos/4.5)**order * np.cos(order*theta)
    if skew == 1:
        b_magnitude = strength * (r_pos/4.5)**order * np.sin(order*theta)
    return b_magnitude

class Geometry:
    def __init__(self, x, y, p):
        self.xpos = x
        self.ypos = y
        self.probes = p.astype('int')
        if len(self.xpos) != len(self.ypos):
            print("In Geometry length of xpos and ypos are not equal")
        elif len(self.ypos) != len(self.probes):
            print("In Geometry length of probes and ypos are not equal")
    def IsReduced(self):
        isReduced = True
        for i in self.probes:
            if i == 0:
                isReduced = i
        return isReduced

In [60]:
def DropPos(geom):
    badProbes = np.array([])
    for i in range(len(geom.probes)):
        if not geom.probes[i]:
            badProbes = np.append(badProbes, i)
    badProbes = badProbes.astype(int)
    X = geom.xpos.copy()
    Y = geom.ypos.copy()
    px_new = np.delete(X, badProbes) # Only the x pos of working probes stay
    py_new = np.delete(Y, badProbes) # Same for y
    PROBES_NEW = np.delete(geom.probes, badProbes) #Now this is an array of 1s.
    reducedGeometry = Geometry(px_new, py_new, PROBES_NEW)
    return reducedGeometry, badProbes

def Theta(geom, badProbes, mult_order, mult_skew):    
    if not geom.IsReduced():
        print("The geometry given has non-working probes")
        return
    lenOrigProbes = len(geom.probes) + len(badProbes)
    _MULTS_FP_N = np.array([__multipole(mult_order[i], mult_skew[i], 1, geom.xpos, geom.ypos)\
                            for i in range(len(mult_order) - len(badProbes))])
    _MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0
    
    INV_MULT = np.linalg.pinv(np.transpose(_MULTS_FP_N))
    INV_MULT[np.abs(INV_MULT) < 1.0e-9] = 0
    
    #Add the columns of 0
    inv_mult = np.zeros((len(INV_MULT), lenOrigProbes))
    j = 0
    if len(badProbes) > 0:
        badProbesCopy = badProbes.copy()
        for i in range(lenOrigProbes):
            if i != badProbesCopy[0]:
                inv_mult[:,i] = INV_MULT[:,j]
                j+=1
            elif (i == badProbesCopy[0]) and (len(badProbesCopy) > 1):
                badProbesCopy = np.delete(badProbesCopy, 0)
                  
        #Add rows at the end, only the first moments can be calculated
        for i in range(lenOrigProbes - len(geom.probes)):
            inv_mult = np.concatenate((inv_mult, np.zeros((1,lenOrigProbes))))
        return(inv_mult)
    else:
        return(INV_MULT)

    
def ProbeDrops(geom):
    reducedGeometry, badProbes = DropPos(geom)# Return another geom
    
    _MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7]
    _MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
    if len(geom.probes) < len(_MULTIPOLE_ORDER):
        _MULTIPOLE_ORDER = _MULTIPOLE_ORDER[:len(geom.probes)]
        _MULTIPOLE_SKEW = _MULTIPOLE_SKEW[:len(geom.probes)]

    return Theta(reducedGeometry, badProbes, _MULTIPOLE_ORDER, _MULTIPOLE_SKEW)

In [66]:
Trolley = Geometry(trfp.TR_X, trfp.TR_Y, np.zeros(17)+1)
print(ProbeDrops(Trolley).shape)

(14, 17)


In [5]:
J_6_PROBE6x6 = np.array([[1.0, 0.0, 0.0, 0.0, 2.631605, 0.0],
                         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0, 0.0, -0.339835],
                         [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 0.0, 0.0, -0.166667]])

J_6_PROBE = np.array([[1.0, 0.0, 0.0, 0.0, 2.631605],
                      [0.0, 1.0, 0.0, 0.0, 0.0],
                      [0.0, 0.0, 1.0, 0.0, 0.0],
                      [0.0, 0.0, 0.0, 1.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 1.0]])

J_6_PROBE_OFFSET = np.array([[1.0, 0.222222, 0.0, 0.0, 2.680987],
                             [0.0, 1.0, 0.0, 0.0, 0.444444],
                             [0.0, 0.0, 1.0, 0.444444, 0.0],
                             [0.0, 0.0, 0.0, 1.0, 0.0],
                             [0.0, 0.0, 0.0, 0.0, 1.0]])

J_4_PROBE = np.array([[1.0, 0.0, 0.0, 0.0, 2.927901],
                      [0.0, 1.0, 0.0, 0.0, -0.666667],
                      [0.0, 0.0, 1.0, -0.666667, 0.0],
                      [0.0, 0.0, 0.0, 1.0, 0.0],
                      [0.0, 0.0, 0.0, 0.0, 1.0]])

J_4_PROBE_ST41 = np.array([[1.0, 0.0, 0.0, 0.0, 2.927901],
                           [0.0, 1.0, 0.0, 0.0, 0.0],
                           [0.0, -0.194805, 1.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 1.0, -0.194805],
                           [0.0, 0.0, 0.0, 0.0, 1.0]])

J_4_PROBE_ST37_ST39 = np.array([[1.0, 0.0, 0.0, 0.0, 2.927901],
                                [0.0, 1.0, 0.0, 0.0, 0.0], 
                                [0.0, -0.194805, 1.0, 0.0, 0.0], 
                                [0.0, 0.0, 0.0, 1.0, -0.194805], 
                                [0.0, 0.0, 0.0, 0.0, 1.0]])

In [6]:
THETA_FP_6 = np.array([np.array([1, 1, 1, 1, 1, 1])/6.,  # dipole
                       np.array([1, 0, -1, 1, 0, -1])/-12.*4.5,  # n quad
                       np.array([1, 1, 1, -1, -1, -1])/46.2*4.5,  # s quad
                       np.array([1, 0, -1, -1, 0, 1])/-92.4/2*4.5**2,  # s sext
                       np.array([1, -2, 1, 1, -2, 1])/18./2*4.5**2,  # n sext
                       np.array([1, -2, 1, -1, 2, -1])/-138.6*4.5**3]  # NOT oct, no idea what this is
                     )

THETA_FP_4 = np.array([np.array([1, 0, 1, 0])/2.,  # dipole
                       np.array([1, -1, 1, -1])/-6.*4.5,  # n quad
                       np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                       np.array([1, -1, -1, 1])/-46.2/2*4.5**2]  # sext
                     )

# four probe stations in the garage (stations 37, 39, 41) have a different arrangement of probes
# STATION 37, 39
# TM, TO, BI, BM
THETA_FP_4_ST37_ST39 = np.array([np.array([1, 0, 0, 1])/2.,  #dipole
                                 np.array([-1, 1, -1, 1])/6.*4.5,  # n quad
                                 np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                                 np.array([-1, 1, 1, -1])/46.2/2*4.5**2]  # sext?
                               )
# STATION 41
# TO, TM, BI, BM
THETA_FP_4_ST41 = np.array([np.array([0, 1, 0, 1])/2.,  #dipole
                            np.array([1, -1, -1, 1])/6.*4.5,  # n quad
                            np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                            np.array([1, -1, 1, -1])/46.2/2*4.5**2]  # sext?
                          )

In [10]:
#Checking what each probe would read given a moment of magnitude 1
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1]

#_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7]
#_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]

_MULTS_FP_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(len(_MULTIPOLE_ORDER) - len(drops))])
_MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0
#print(_MULTS_FP_N)

THETA_FPN = np.linalg.pinv(np.transpose(_MULTS_FP_N))
THETA_FPN[np.abs(THETA_FPN) < 1.0e-9] = 0
print(THETA_FPN)
#print(THETA_FP_6)

[[ 1.64694444 -2.79388889  1.64694444  1.64694444 -2.79388889  1.64694444]
 [-0.375       0.          0.375      -0.375       0.          0.375     ]
 [ 0.32083333 -0.34945887  0.32083333 -0.32083333  0.34945887 -0.32083333]
 [-0.10957792  0.          0.10957792  0.10957792  0.         -0.10957792]
 [ 0.5625     -1.125       0.5625      0.5625     -1.125       0.5625    ]
 [ 0.10957792 -0.21915584  0.10957792 -0.10957792  0.21915584 -0.10957792]]


The above code works for 6 working fp. Below will be testing for 4 probes

In [ ]:
#### For some reason this changes when more multipoles are used. I am assuming it has to do with m6 not bing linearly indep.
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 0]
_MULTS_FP6 = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X, FP6_Y) for i in range(len(_MULTIPOLE_ORDER))])
_MULTS_FP6[np.abs(_MULTS_FP6) < 1.0e-9] = 0

THETA_FP6 = np.linalg.pinv(np.transpose(_MULTS_FP6))
THETA_FP6[np.abs(THETA_FP6) < 1.0e-9] = 0
print(THETA_FP6)

In [ ]:
######### Below is an attempt to create a station with 4 probes

In [ ]:
PROBES = [0,1,1,0,1,1] #1 means the probe is working properly, 0, it is dropped or does not exist.
drops = np.array([])
for i in range(len(PROBES)):
    if not PROBES[i]:
        drops = np.append(drops, i)
drops = drops.astype(int)


FP6_X_NEW = np.delete(FP6_X, drops)  # now only the x pos of working probes are stored
FP6_Y_NEW = np.delete(FP6_Y, drops)  # same with y pos
PROBES_NEW = np.delete(PROBES, drops)# array of 1s of length of the number of working probes

In [ ]:
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1]

_MULTS_FP_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, FP6_X_NEW, FP6_Y_NEW) for i in range(len(_MULTIPOLE_ORDER) - len(drops))])
_MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0

INV_MULT = np.linalg.pinv(np.transpose(_MULTS_FP_N))
INV_MULT[np.abs(INV_MULT) < 1.0e-9] = 0

#Add the columns of 0s
inv_mult = np.zeros((len(INV_MULT), len(PROBES)))
j = 0
for i in range(len(PROBES)):
    if PROBES[i] == 1:
        inv_mult[:,i] = INV_MULT[:,j]
        j+=1
#Add rows
for i in range(len(PROBES) - len(PROBES_NEW)):
    inv_mult = np.concatenate((inv_mult, np.zeros((1,len(PROBES)))))

THETA = np.dot(np.linalg.pinv(J_6_PROBE6x6), inv_mult)
THETA[np.abs(THETA) < 1.0e-9] = 0
THETA = THETA[:len(PROBES_NEW)]
T = np.zeros((len(PROBES_NEW),len(PROBES_NEW)))
j = 0
for i in range(len(PROBES)):
    if(np.sum(np.abs(THETA[:,i])))> 1e-8:
        T[:,j] = THETA[:,i]
        j+=1
THETA = T
print("Calculated Theta for 4 fp stattion")
print(THETA)
print("############")
print("Hard Coded theta for 4 fp station")
print(np.array([np.array([1, 0, 1, 0])/2.,  # dipole
                       np.array([1, -1, 1, -1])/-6.*4.5,  # n quad
                       np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                       np.array([1, -1, -1, 1])/-46.2/2*4.5**2]  # sext
                     )
)

In [ ]:
####### Now trolley

In [35]:
PROBES = np.zeros(17) + 1 #1 means the probe is working properly, 0, it is dropped or does not exist.
PROBES = PROBES.astype('int')
PROBES[8] = 0
print(PROBES)
drops = np.array([]) #This is meant to be empty, will be filled to know what probes to drop
for i in range(len(PROBES)):
    if not PROBES[i]:
        drops = np.append(drops, i)
drops = drops.astype(int)

X = trfp.TR_X.copy()
Y = trfp.TR_Y.copy()

TR_X_NEW = np.delete(X, drops)  # now only the x pos of working probes are stored
TR_Y_NEW = np.delete(Y, drops)  # same with y pos
PROBES_NEW = np.delete(PROBES, drops)# array of 1s of length of the number of working probes

[1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]


In [36]:
_MULTIPOLE_ORDER = [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7]
_MULTIPOLE_SKEW = [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1]
# Skew 14 pole is unmeasurable probes are on nodes.
# 4  is 10 pole, anything above this is unreliable. Uncertainty on cal. coeff. --> prop. uncertainty...
# on cal. becomes too large

if len(drops) == 0:
    _MULTS_TR_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, TR_X_NEW, TR_Y_NEW) for i in range(len(_MULTIPOLE_ORDER))])

else: #This else must be corrected for trolley drops instead of fp
    _MULTS_TR_N = np.array([__multipole(_MULTIPOLE_ORDER[i], _MULTIPOLE_SKEW[i], 1, TR_X_NEW, TR_Y_NEW) for i in range(len(_MULTIPOLE_ORDER) - len(drops) + 1)])

_MULTS_TR_N[np.abs(_MULTS_TR_N) < 1.0e-9] = 0 


INV_MULT = np.linalg.pinv(np.transpose(_MULTS_TR_N))
INV_MULT[np.abs(INV_MULT) < 1.0e-9] = 0
#Add the columns of 0s
inv_mult = np.zeros((len(INV_MULT), len(PROBES)))
j = 0
for i in range(len(PROBES)):
    if PROBES[i] == 1:
        inv_mult[:,i] = INV_MULT[:,j]
        j+=1
for i in range(len(PROBES) - len(PROBES_NEW)):
    inv_mult = np.concatenate((inv_mult, np.zeros((1,len(PROBES)))))
    
print(inv_mult)

[[ 1.45371227e-01  9.76731955e-02  2.24179677e-01  9.76731955e-02
   2.24179677e-01  2.17433015e-02  2.58712125e-02  1.59878936e-02
   0.00000000e+00  1.59878936e-02  2.58712125e-02  2.17433015e-02
   2.58712125e-02  1.59878936e-02  0.00000000e+00  1.59878936e-02
   2.58712125e-02]
 [ 3.28613935e-01  1.48014153e-01  6.28547775e-01  1.48014153e-01
   6.28547775e-01 -1.31693998e-01 -2.28085941e-02  1.80837317e-02
   0.00000000e+00  1.80837317e-02 -2.28085941e-02 -1.31693998e-01
  -2.37094308e-01 -3.53070013e-01 -4.28571429e-01 -3.53070013e-01
  -2.37094308e-01]
 [ 0.00000000e+00 -9.83984625e-02  0.00000000e+00  9.83984625e-02
   0.00000000e+00 -1.92761051e-01 -1.72261940e-01 -1.02530429e-01
   0.00000000e+00  1.02530429e-01  1.72261940e-01  1.92761051e-01
   1.72261940e-01  1.02530429e-01  0.00000000e+00 -1.02530429e-01
  -1.72261940e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.38598836e-01 -2.38598836e-01
   0.00000000e+00  